# Quickstart notebook
The example code here shows how to get up and running with Mosaic using the Python API.

In [0]:
from pyspark.sql.functions import *

## Enable Mosaic in the notebook
To get started, you'll need to attach the python library to your cluster and execute the `enable_mosaic` function.

In [0]:
from mosaic import enable_mosaic
enable_mosaic(spark, dbutils)

Mosaic has extra configuration options. Check the docs for more details.

In [0]:
help(enable_mosaic)

Help on function enable_mosaic in module mosaic.api.enable:

enable_mosaic(spark: pyspark.sql.session.SparkSession, dbutils=None) -> None
 Enable Mosaic functions.
 
 Use this function at the start of your workflow to ensure all of the required dependencies are installed and
 Mosaic is configured according to your needs.
 
 Parameters
 ----------
 spark : pyspark.sql.SparkSession
 The active SparkSession.
 dbutils : dbruntime.dbutils.DBUtils
 The dbutils object used for `display` and `displayHTML` functions.
 Optional, only applicable to Databricks users.
 
 Returns
 -------
 
 Notes
 -----
 Users can control various aspects of Mosaic's operation with the following Spark confs:
 
 - `spark.databricks.labs.mosaic.jar.autoattach`: 'true' (default) or 'false'
 Automatically attach the Mosaic JAR to the Databricks cluster? (Optional)
 - `spark.databricks.labs.mosaic.jar.location`
 Explicitly specify the path to the Mosaic JAR.
 (Optional and not required at all in a standard Databricks environment).
 - `spark.databricks.labs.mosaic.geometry.api`: 'OGC' (default) or 'JTS'
 Explicitly specify the underlying geometry library to use for spatial operations. (Optional)
 - `spark.databricks.labs.mosaic.index.system`: 'H3' (default)
 Explicitly specify the index system to use for optimized spatial joins. (Optional)

## Geometry constructors and the Mosaic internal geometry format

Mosaic allows users to create new Point geometries from a pair of Spark DoubleType columns.

In [0]:
from mosaic import st_point

lons = [-80., -80., -70., -70., -80.]
lats = [ 35.,  45.,  45.,  35.,  35.]

bounds_df = (
  spark
  .createDataFrame({"lon": lon, "lat": lat} for lon, lat in zip(lons, lats))
  .coalesce(1)
  .withColumn("point_geom", st_point("lon", "lat"))
)
bounds_df.show()

+----+-----+--------------------+
 lat| lon| point_geom|
+----+-----+--------------------+
35.0|-80.0|{1, [[[-80.0, 35....|
45.0|-80.0|{1, [[[-80.0, 45....|
45.0|-70.0|{1, [[[-70.0, 45....|
35.0|-70.0|{1, [[[-70.0, 35....|
35.0|-80.0|{1, [[[-80.0, 35....|
+----+-----+--------------------+

Mosaic Point geometries can be aggregated into LineString and Polygon geometries using the respective constructors.

In [0]:
from mosaic import st_makeline

bounds_df = (
  bounds_df
  .groupBy()
  .agg(collect_list("point_geom").alias("bounding_coords"))
  .select(st_makeline("bounding_coords").alias("bounding_ring"))
)
bounds_df.show()

+--------------------+
 bounding_ring|
+--------------------+
{3, [[[-80.0, 35....|
+--------------------+

In [0]:
from mosaic import st_makepolygon

bounds_df = bounds_df.select(st_makepolygon("bounding_ring").alias("bounds"))
bounds_df.show()

+--------------------+
 bounds|
+--------------------+
{5, [[[-80.0, 35....|
+--------------------+

## Geometry clipping without an index

Mosaic implements set intersection functions: contains, intersects, overlaps etc. Here you can see `st_contains` being used to clip points by a polygon geometry.

In [0]:
tripsTable = spark.table("delta.`/databricks-datasets/nyctaxi/tables/nyctaxi_yellow`")

In [0]:
from mosaic import st_contains
trips = (
  tripsTable
  .limit(5_000_000)
  .repartition(sc.defaultParallelism * 20)
  .drop("vendorId", "rateCodeId", "store_and_fwd_flag", "payment_type")
  .withColumn("pickup_geom", st_point("pickup_longitude", "pickup_latitude"))
  .withColumn("dropoff_geom", st_point("dropoff_longitude", "dropoff_latitude"))
  .crossJoin(bounds_df)
  .where(st_contains("bounds", "pickup_geom"))
  .where(st_contains("bounds", "dropoff_geom"))
  .cache()
)

In [0]:
trips.show()

+---------+-------------------+-------------------+---------------+-------------+----------------+---------------+------------+-----------------+----------------+-----------+-----+-------+----------+------------+------------+--------------------+--------------------+--------------------+
vendor_id| pickup_datetime| dropoff_datetime|passenger_count|trip_distance|pickup_longitude|pickup_latitude|rate_code_id|dropoff_longitude|dropoff_latitude|fare_amount|extra|mta_tax|tip_amount|tolls_amount|total_amount| pickup_geom| dropoff_geom| bounds|
+---------+-------------------+-------------------+---------------+-------------+----------------+---------------+------------+-----------------+----------------+-----------+-----+-------+----------+------------+------------+--------------------+--------------------+--------------------+
 CMT|2009-10-12 19:35:21|2009-10-12 19:44:36| 1| 1.4| -74.000018| 40.720534| null| -73.995008| 40.734112| 6.5| 0.0| null| 0.0| 0.0| 6.5|{1, [[[-74.000018...|{1, [[[-73.995008...|{5, [[[-80.0, 35....|
 CMT|2010-09-02 07:37:40|2010-09-02 07:53:17| 1| 6.8| -74.016477| 40.70971| 1| -73.97509| 40.759392| 17.3| 0.0| 0.5| 1.0| 0.0| 18.8|{1, [[[-74.016477...|{1, [[[-73.97509,...|{5, [[[-80.0, 35....|
 CMT|2009-11-28 13:00:00|2009-11-28 13:01:57| 1| 0.5| -73.999666| 40.733384| null| -73.991751| 40.730273| 3.3| 0.0| 0.5| 0.0| 0.0| 3.8|{1, [[[-73.999666...|{1, [[[-73.991751...|{5, [[[-80.0, 35....|
 CMT|2009-11-30 07:49:13|2009-11-30 08:12:38| 1| 12.9| -74.007033| 40.734931| null| -73.87277| 40.774319| 29.7| 0.0| 0.5| 6.04| 0.0| 36.24|{1, [[[-74.007033...|{1, [[[-73.87277,...|{5, [[[-80.0, 35....|
 VTS|2009-01-03 01:55:00|2009-01-03 02:04:00| 5| 2.09| -73.993217| 40.72416| null| -73.9944| 40.74624| 7.7| 0.5| null| 0.0| 0.0| 8.2|{1, [[[-73.993217...|{1, [[[-73.9944, ...|{5, [[[-80.0, 35....|
 VTS|2009-11-09 17:21:00|2009-11-09 17:32:00| 1| 5.19| -74.010003| 40.707022| null| -73.963853| 40.75677| 13.3| 1.0| 0.5| 0.0| 0.0| 14.8|{1, [[[-74.010003...|{1, [[[-73.963853...|{5, [[[-80.0, 35....|
 VTS|2010-06-09 01:13:00|2010-06-09 01:27:00| 1| 8.09| -74.008697| 40.707335| 1| -73.949605| 40.78761| 19.7| 0.5| 0.5| 0.0| 0.0| 20.7|{1, [[[-74.008697...|{1, [[[-73.949605...|{5, [[[-80.0, 35....|
 VTS|2009-01-28 18:36:00|2009-01-28 18:50:00| 1| 2.92| -74.008823| 40.72089| null| -73.994055| 40.747588| 10.1| 1.0| null| 3.0| 0.0| 14.1|{1, [[[-74.008823...|{1, [[[-73.994055...|{5, [[[-80.0, 35....|
 CMT|2010-06-02 14:21:05|2010-06-02 14:28:43| 1| 5.0| -74.011653| 40.702604| 1| -73.971851| 40.749807| 12.5| 0.0| 0.5| 2.6| 0.0| 15.6|{1, [[[-74.011653...|{1, [[[-73.971851...|{5, [[[-80.0, 35....|
 VTS|2009-11-26 15:38:00|2009-11-26 15:47:00| 5| 2.36| -73.997208| 40.722108| null| -73.99647| 40.748185| 8.1| 0.0| 0.5| 2.0| 0.0| 10.6|{1, [[[-73.997208...|{1, [[[-73.99647,...|{5, [[[-80.0, 35....|
 CMT|2009-12-07 10:36:06|2009-12-07 10:42:26| 1| 1.3| -73.99067| 40.73065| null| -73.975539| 40.741353| 6.1| 0.0| 0.5| 0.0| 0.0| 6.6|{1, [[[-73.99067,...|{1, [[[-73.975539...|{5, [[[-80.0, 35....|
 CMT|2009-11-15 09:11:42|2009-11-15 09:26:34| 1| 3.7| -73.990065| 40.687341| null| -73.999566| 40.730219| 12.1| 0.0| 0.5| 0.0| 0.0| 12.6|{1, [[[-73.990065...|{1, [[[-73.999566...|{5, [[[-80.0, 35....|
 CMT|2010-06-27 17:27:25|2010-06-27 17:47:44| 2| 5.0| -74.008459| 40.712233| 1| -73.9516| 40.718877| 15.7| 0.0| 0.5| 3.0| 0.0| 19.2|{1, [[[-74.008459...|{1, [[[-73.9516, ...|{5, [[[-80.0, 35....|
 VTS|2009-01-30 21:05:00|2009-01-30 21:34:00| 1| 5.99| -73.994397| 40.694843| null| -73.921257| 40.685107| 18.9| 0.5| null| 0.6| 0.0| 20.0|{1, [[[-73.994397...|{1, [[[-73.921257...|{5, [[[-80.0, 35....|
 CMT|2009-01-14 07:43:53|2009-01-14 07:55:14| 2| 2.9| -74.007602| 40.725098| null| -73.977846| 40.751608| 9.3| 0.0| null| 0.0| 0.0| 9.3|{1, [[[-74.007602...|{1, [[[-73.977846...|{5, [[[-80.0, 35....|
 VTS|2010-05-11 16:33:00|2010-05-11 16:41:00| 1| 0.99| -74.011205| 40.715468| 1| -74.011072| 40.708882| 5.7| 1.0| 0.5| 0.0| 0.0| 7.2|{1, [[[-74.011205...|{1, [[[-74.011072...|{

## Read from GeoJson, compute some basic geometry attributes

You've seen how Mosaic can create geometries from Spark native data types but it also provides functions to translate Well Known Text (WKT), Well Known Binary (WKB) and GeoJSON representations to Mosaic geometries.

In [0]:
from mosaic import st_geomfromgeojson

geoJsonDF = (
  spark.read.format("json")
  .load("dbfs:/FileStore/shared_uploads/stuart.lynn@databricks.com/NYC_Taxi_Zones.geojson")
  .withColumn("geometry", st_geomfromgeojson(to_json(col("geometry"))))
  .select("properties.*", "geometry")
  .drop("shape_area", "shape_leng")
)

geoJsonDF.show()

+-------------+-----------+--------+--------------------+--------------------+
 borough|location_id|objectid| zone| geometry|
+-------------+-----------+--------+--------------------+--------------------+
 EWR| 1| 1| Newark Airport|{6, [[[-74.184452...|
 Queens| 2| 2| Jamaica Bay|{6, [[[-73.823375...|
 Bronx| 3| 3|Allerton/Pelham G...|{6, [[[-73.847926...|
 Manhattan| 4| 4| Alphabet City|{6, [[[-73.971774...|
Staten Island| 5| 5| Arden Heights|{6, [[[-74.174217...|
Staten Island| 6| 6|Arrochar/Fort Wad...|{6, [[[-74.063673...|
 Queens| 7| 7| Astoria|{6, [[[-73.904136...|
 Queens| 8| 8| Astoria Park|{6, [[[-73.923340...|
 Queens| 9| 9| Auburndale|{6, [[[-73.785024...|
 Manhattan| 24| 24| Bloomingdale|{6, [[[-73.959536...|
 Queens| 10| 10| Baisley Park|{6, [[[-73.783266...|
 Brooklyn| 11| 11| Bath Beach|{6, [[[-74.001098...|
 Manhattan| 12| 12| Battery Park|{6, [[[-74.015657...|
 Manhattan| 13| 13| Battery Park City|{6, [[[-74.012441...|
 Bronx| 18| 18| Bedford Park|{6, [[[-73.885139...|
 Brooklyn| 25| 25| Boerum Hill|{6, [[[-73.981552...|
 Brooklyn| 14| 14| Bay Ridge|{6, [[[-74.034073...|
 Queens| 15| 15|Bay Terrace/Fort ...|{6, [[[-73.777403...|
 Brooklyn| 22| 22| Bensonhurst West|{6, [[[-73.992549...|
Staten Island| 23| 23|Bloomfield/Emerso...|{6, [[[-74.195686...|
+-------------+-----------+--------+--------------------+--------------------+
only showing top 20 rows

Mosaic provides a number of functions for extracting the properties of geometries. Here are some that are relevant to Polygon geometries:

In [0]:
from mosaic import st_area, st_length
(
  geoJsonDF
  .withColumn("calculatedArea", abs(st_area("geometry")))
  .withColumn("calculatedLength", st_length("geometry"))
  .select("geometry", "calculatedArea", "calculatedLength")
).show()

+--------------------+--------------------+--------------------+
 geometry| calculatedArea| calculatedLength|
+--------------------+--------------------+--------------------+
{6, [[[-74.184452...|7.823067885002562E-4| 0.1163574531886787|
{6, [[[-73.823375...|0.001422779097814599| 0.8431218810128789|
{6, [[[-73.847926...|3.144141568206508E-4| 0.08434110590105784|
{6, [[[-73.971774...| 7.94539194214528E-5| 0.03559982116907461|
{6, [[[-74.174217...|4.979574893632193E-4| 0.09214648985735088|
{6, [[[-74.063673...|4.046076598541714E-4| 0.12905753305870837|
{6, [[[-73.904136...|3.897879892739018...| 0.10741717112348065|
{6, [[[-73.923340...|2.658771690499072E-5|0.027590691195158792|
{6, [[[-73.785024...|3.384438031986125...| 0.0997840924710119|
{6, [[[-73.959536...|4.193691000471793E-5|0.034022318280805826|
{6, [[[-73.783266...|4.358238180810105...| 0.09983947941552027|
{6, [[[-74.001098...|2.029913532381591E-4| 0.07243934539937844|
{6, [[[-74.015657...|1.078595394871048...| 0.01759380786230351|
{6, [[[-74.012441...|4.874002713725957E-5| 0.04726120195723124|
{6, [[[-73.885139...|1.488501639473639E-4| 0.06979954985700013|
{6, [[[-73.981552...|1.241682673558662...| 0.04714581993191898|
{6, [[[-74.034073...|6.636590898190788E-4| 0.13622530962992194|
{6, [[[-73.777403...|4.588438269978550...| 0.1295148763926726|
{6, [[[-73.992549...|4.613649657988681...| 0.11844001178496774|
{6, [[[-74.195686...|0.002090822778761513| 0.3007595308122533|
+--------------------+--------------------+--------------------+
only showing top 20 rows

In [0]:
geoJsonDF.count()

Out[12]: 263

## Example point-in-poly with indexing

Mosaic has built-in support for the popular spatial indexing library, H3. The user has access to functions for generating point indices and the sets of indices covering polygons, allowing point-in-polygon joins to be transformed into deterministic SQL joins.

In [0]:
from mosaic import grid_longlatascellid

trips_with_geom = (
  trips
  .withColumn("pickup_h3", grid_longlatascellid(lon="pickup_longitude", lat="pickup_latitude", resolution=lit(10)))
  .withColumn("dropoff_h3", grid_longlatascellid(lon="dropoff_longitude", lat="dropoff_latitude", resolution=lit(10)))
)

trips_with_geom.show()

+---------+-------------------+-------------------+---------------+-------------+----------------+---------------+------------+-----------------+----------------+-----------+-----+-------+----------+------------+------------+--------------------+--------------------+--------------------+------------------+------------------+
vendor_id| pickup_datetime| dropoff_datetime|passenger_count|trip_distance|pickup_longitude|pickup_latitude|rate_code_id|dropoff_longitude|dropoff_latitude|fare_amount|extra|mta_tax|tip_amount|tolls_amount|total_amount| pickup_geom| dropoff_geom| bounds| pickup_h3| dropoff_h3|
+---------+-------------------+-------------------+---------------+-------------+----------------+---------------+------------+-----------------+----------------+-----------+-----+-------+----------+------------+------------+--------------------+--------------------+--------------------+------------------+------------------+
 CMT|2009-10-12 19:35:21|2009-10-12 19:44:36| 1| 1.4| -74.000018| 40.720534| null| -73.995008| 40.734112| 6.5| 0.0| null| 0.0| 0.0| 6.5|{1, [[[-74.000018...|{1, [[[-73.995008...|{5, [[[-80.0, 35....|622236750705885183|622236723432488959|
 CMT|2010-09-02 07:37:40|2010-09-02 07:53:17| 1| 6.8| -74.016477| 40.70971| 1| -73.97509| 40.759392| 17.3| 0.0| 0.5| 1.0| 0.0| 18.8|{1, [[[-74.016477...|{1, [[[-73.97509,...|{5, [[[-80.0, 35....|622236750651359231|622236723493961727|
 CMT|2009-11-28 13:00:00|2009-11-28 13:01:57| 1| 0.5| -73.999666| 40.733384| null| -73.991751| 40.730273| 3.3| 0.0| 0.5| 0.0| 0.0| 3.8|{1, [[[-73.999666...|{1, [[[-73.991751...|{5, [[[-80.0, 35....|622236723433046015|622236750715092991|
 CMT|2009-11-30 07:49:13|2009-11-30 08:12:38| 1| 12.9| -74.007033| 40.734931| null| -73.87277| 40.774319| 29.7| 0.0| 0.5| 6.04| 0.0| 36.24|{1, [[[-74.007033...|{1, [[[-73.87277,...|{5, [[[-80.0, 35....|622236750719975423|622236724015759359|
 VTS|2009-01-03 01:55:00|2009-01-03 02:04:00| 5| 2.09| -73.993217| 40.72416| null| -73.9944| 40.74624| 7.7| 0.5| null| 0.0| 0.0| 8.2|{1, [[[-73.993217...|{1, [[[-73.9944, ...|{5, [[[-80.0, 35....|622236750717878271|622236723431276543|
 VTS|2009-11-09 17:21:00|2009-11-09 17:32:00| 1| 5.19| -74.010003| 40.707022| null| -73.963853| 40.75677| 13.3| 1.0| 0.5| 0.0| 0.0| 14.8|{1, [[[-74.010003...|{1, [[[-73.963853...|{5, [[[-80.0, 35....|622236750647689215|622236723504381951|
 VTS|2010-06-09 01:13:00|2010-06-09 01:27:00| 1| 8.09| -74.008697| 40.707335| 1| -73.949605| 40.78761| 19.7| 0.5| 0.5| 0.0| 0.0| 20.7|{1, [[[-74.008697...|{1, [[[-73.949605...|{5, [[[-80.0, 35....|622236750647820287|622236722210668543|
 VTS|2009-01-28 18:36:00|2009-01-28 18:50:00| 1| 2.92| -74.008823| 40.72089| null| -73.994055| 40.747588| 10.1| 1.0| null| 3.0| 0.0| 14.1|{1, [[[-74.008823...|{1, [[[-73.994055...|{5, [[[-80.0, 35....|622236750713159679|622236723439304703|
 CMT|2010-06-02 14:21:05|2010-06-02 14:28:43| 1| 5.0| -74.011653| 40.702604| 1| -73.971851| 40.749807| 12.5| 0.0| 0.5| 2.6| 0.0| 15.6|{1, [[[-74.011653...|{1, [[[-73.971851...|{5, [[[-80.0, 35....|622236750639857663|622236723496026111|
 VTS|2009-11-26 15:38:00|2009-11-26 15:47:00| 5| 2.36| -73.997208| 40.722108| null| -73.99647| 40.748185| 8.1| 0.0| 0.5| 2.0| 0.0| 10.6|{1, [[[-73.997208...|{1, [[[-73.99647,...|{5, [[[-80.0, 35....|622236750706966527|622236723431309311|
 CMT|2009-12-07 10:36:06|2009-12-07 10:42:26| 1| 1.3| -73.99067| 40.73065| null| -73.975539| 40.741353| 6.1| 0.0| 0.5| 0.0| 0.0| 6.6|{1, [[[-73.99067,...|{1, [[[-73.975539...|{5, [[[-80.0, 35....|622236750715092991|622236723437502463|
 CMT|2009-11-15 09:11:42|2009-11-15 09:26:34| 1| 3.7| -73.990065| 40.687341| null| -73.999566| 40.730219| 12.1| 0.0| 0.5| 0.0| 0.0| 12.6|{1, [[[-73.990065...|{1, [[[-73.999566...|{5, [[[-80.0, 35....|622236750667481087|622236750719549439|
 CMT|2010-06-27 17:27:25|2010-06-27 17:47:44| 2| 5.0| -74.008459| 40.712233| 1| -73.9516| 40.718877| 15.7| 0.0| 0.5| 3.0| 0.0| 19.2|{1, [[[-74.008459...|{1, [[[-73.9516, ...|{5, [[[-80.0, 35....|622236750648639487|622236

In [0]:
from mosaic import grid_polyfill

neighbourhoods = (
  geoJsonDF
  .repartition(sc.defaultParallelism)
  .select("*", explode(grid_polyfill("geometry", lit(10))).alias("h3"))
  .drop("geometry")
)

neighbourhoods.show()

+-------------+-----------+--------+---------------+------------------+
 borough|location_id|objectid| zone| h3|
+-------------+-----------+--------+---------------+------------------+
Staten Island| 99| 99|Freshkills Park|622236745731309567|
Staten Island| 99| 99|Freshkills Park|622236745690841087|
Staten Island| 99| 99|Freshkills Park|622236745723674623|
Staten Island| 99| 99|Freshkills Park|622236745683206143|
Staten Island| 99| 99|Freshkills Park|622236745691987967|
Staten Island| 99| 99|Freshkills Park|622236745684353023|
Staten Island| 99| 99|Freshkills Park|622236745733603327|
Staten Island| 99| 99|Freshkills Park|622236745725968383|
Staten Island| 99| 99|Freshkills Park|622236745701916671|
Staten Island| 99| 99|Freshkills Park|622236745734750207|
Staten Island| 99| 99|Freshkills Park|622236745694281727|
Staten Island| 99| 99|Freshkills Park|622236745703063551|
Staten Island| 99| 99|Freshkills Park|622236745679011839|
Staten Island| 99| 99|Freshkills Park|622236745761095679|
Staten Island| 99| 99|Freshkills Park|622236745680158719|
Staten Island| 99| 99|Freshkills Park|622236745696575487|
Staten Island| 99| 99|Freshkills Park|622236745762242559|
Staten Island| 99| 99|Freshkills Park|622236745681305599|
Staten Island| 99| 99|Freshkills Park|622236745730555903|
Staten Island| 99| 99|Freshkills Park|622236745690087423|
+-------------+-----------+--------+---------------+------------------+
only showing top 20 rows

In [0]:
joined_df = trips_with_geom.alias("t").join(neighbourhoods.alias("n"), on=expr("t.pickup_h3 = n.h3"), how="inner")
joined_df.count()

Out[15]: 4934151

## Mosaic spatial join optimizations

Mosaic provides easy access to the optimized spatial join technique described in [this](https://databricks.com/blog/2021/10/11/efficient-point-in-polygon-joins-via-pyspark-and-bng-geospatial-indexing.html) blog post.

In [0]:
from mosaic import grid_tessellateexplode

mosaic_neighbourhoods = (
  geoJsonDF
  .repartition(sc.defaultParallelism)
  .select("*", grid_tessellateexplode("geometry", lit(10)))
  .drop("geometry")
)

mosaic_neighbourhoods.show()

+-------------+-----------+--------+---------------+-------+------------------+----+
 borough|location_id|objectid| zone|is_core| h3| wkb|
+-------------+-----------+--------+---------------+-------+------------------+----+
Staten Island| 99| 99|Freshkills Park| true|622236745694314495|null|
Staten Island| 99| 99|Freshkills Park| true|622236745679077375|null|
Staten Island| 99| 99|Freshkills Park| true|622236745734160383|null|
Staten Island| 99| 99|Freshkills Park| true|622236745762275327|null|
Staten Island| 99| 99|Freshkills Park| true|622236745731801087|null|
Staten Island| 99| 99|Freshkills Park| true|622236745725935615|null|
Staten Island| 99| 99|Freshkills Park| true|622236745680224255|null|
Staten Island| 99| 99|Freshkills Park| true|622236745689595903|null|
Staten Island| 99| 99|Freshkills Park| true|622236745683730431|null|
Staten Island| 99| 99|Freshkills Park| true|622236745723576319|null|
Staten Island| 99| 99|Freshkills Park| true|622236745763422207|null|
Staten Island| 99| 99|Freshkills Park| true|622236745693102079|null|
Staten Island| 99| 99|Freshkills Park| true|622236745732947967|null|
Staten Island| 99| 99|Freshkills Park| true|622236745702473727|null|
Staten Island| 99| 99|Freshkills Park| true|622236745696608255|null|
Staten Island| 99| 99|Freshkills Park| true|622236745736454143|null|
Staten Island| 99| 99|Freshkills Park| true|622236745761062911|null|
Staten Island| 99| 99|Freshkills Park| true|622236745690742783|null|
Staten Island| 99| 99|Freshkills Park| true|622236745675505663|null|
Staten Island| 99| 99|Freshkills Park| true|622236745730588671|null|
+-------------+-----------+--------+---------------+-------+------------------+----+
only showing top 20 rows

Mosaic also includes a convenience function for displaying dataframes with geometry columns.

In [0]:
from mosaic import displayMosaic
displayMosaic(mosaic_neighbourhoods)

borough,location_id,objectid,zone,is_core,h3,wkb
Staten Island,99,99,Freshkills Park,true,622236745694314495,null
Staten Island,99,99,Freshkills Park,true,622236745679077375,null
Staten Island,99,99,Freshkills Park,true,622236745734160383,null
Staten Island,99,99,Freshkills Park,true,622236745762275327,null
Staten Island,99,99,Freshkills Park,true,622236745731801087,null
Staten Island,99,99,Freshkills Park,true,622236745725935615,null
Staten Island,99,99,Freshkills Park,true,622236745680224255,null
Staten Island,99,99,Freshkills Park,true,622236745689595903,null
Staten Island,99,99,Freshkills Park,true,622236745683730431,null
Staten Island,99,99,Freshkills Park,true,622236745723576319,null


This also extends to plotting maps inside the notebook using the kepler.gl visualisation library using a notebook magic `%%mosaic_kepler`.

In [0]:
from mosaic import st_aswkt
(
  mosaic_neighbourhoods
  .select(st_aswkt(col("index.wkb")).alias("wkt"), col("index.index_id").alias("h3"))
).createOrReplaceTempView("kepler_df")

In [0]:
%%mosaic_kepler
"kepler_df" "h3" "h3"

User Guide: https://docs.kepler.gl/docs/keplergl-jupyter

<!doctype html> Kepler.gl

![mosaic kepler map example](../images/kepler-example.png)

Now the two datasets can be joined first on H3 index, with any false positives removed through a contains filter on a much simpler geometry.

In [0]:
mosaic_joined_df = (
  trips_with_geom.alias("t")
  .join(mosaic_neighbourhoods.alias("n"), on=expr("t.pickup_h3 = n.index.index_id"), how="inner")
  .where(
    ~col("index.is_core") | 
    st_contains("index.wkb", "pickup_geom")
  )
)

mosaic_joined_df.show()

+---------+-------------------+-------------------+---------------+-------------+----------------+---------------+------------+-----------------+----------------+-----------+-----+-------+----------+------------+------------+--------------------+--------------------+--------------------+------------------+------------------+---------+-----------+--------+--------------------+-------+------------------+--------------------+
vendor_id| pickup_datetime| dropoff_datetime|passenger_count|trip_distance|pickup_longitude|pickup_latitude|rate_code_id|dropoff_longitude|dropoff_latitude|fare_amount|extra|mta_tax|tip_amount|tolls_amount|total_amount| pickup_geom| dropoff_geom| bounds| pickup_h3| dropoff_h3| borough|location_id|objectid| zone|is_core| h3| wkb|
+---------+-------------------+-------------------+---------------+-------------+----------------+---------------+------------+-----------------+----------------+-----------+-----+-------+----------+------------+------------+--------------------+--------------------+--------------------+------------------+------------------+---------+-----------+--------+--------------------+-------+------------------+--------------------+
 CMT|2009-11-28 13:00:00|2009-11-28 13:01:57| 1| 0.5| -73.999666| 40.733384| null| -73.991751| 40.730273| 3.3| 0.0| 0.5| 0.0| 0.0| 3.8|{1, [[[-73.999666...|{1, [[[-73.991751...|{5, [[[-80.0, 35....|622236723433046015|622236750715092991|Manhattan| 114| 114|Greenwich Village...| false|622236723433046015|[01 03 00 00 00 0...|
 CMT|2009-11-28 13:00:00|2009-11-28 13:01:57| 1| 0.5| -73.999666| 40.733384| null| -73.991751| 40.730273| 3.3| 0.0| 0.5| 0.0| 0.0| 3.8|{1, [[[-73.999666...|{1, [[[-73.991751...|{5, [[[-80.0, 35....|622236723433046015|622236750715092991|Manhattan| 249| 249| West Village| false|622236723433046015|[01 03 00 00 00 0...|
 CMT|2009-11-28 13:00:00|2009-11-28 13:01:57| 1| 0.5| -73.999666| 40.733384| null| -73.991751| 40.730273| 3.3| 0.0| 0.5| 0.0| 0.0| 3.8|{1, [[[-73.999666...|{1, [[[-73.991751...|{5, [[[-80.0, 35....|622236723433046015|622236750715092991|Manhattan| 113| 113|Greenwich Village...| false|622236723433046015|[01 03 00 00 00 0...|
 CMT|2009-11-30 07:49:13|2009-11-30 08:12:38| 1| 12.9| -74.007033| 40.734931| null| -73.87277| 40.774319| 29.7| 0.0| 0.5| 6.04| 0.0| 36.24|{1, [[[-74.007033...|{1, [[[-73.87277,...|{5, [[[-80.0, 35....|622236750719975423|622236724015759359|Manhattan| 249| 249| West Village| false|622236750719975423|[01 03 00 00 00 0...|
 CMT|2009-11-30 07:49:13|2009-11-30 08:12:38| 1| 12.9| -74.007033| 40.734931| null| -73.87277| 40.774319| 29.7| 0.0| 0.5| 6.04| 0.0| 36.24|{1, [[[-74.007033...|{1, [[[-73.87277,...|{5, [[[-80.0, 35....|622236750719975423|622236724015759359|Manhattan| 158| 158|Meatpacking/West ...| false|622236750719975423|[01 03 00 00 00 0...|
 VTS|2009-01-03 01:55:00|2009-01-03 02:04:00| 5| 2.09| -73.993217| 40.72416| null| -73.9944| 40.74624| 7.7| 0.5| null| 0.0| 0.0| 8.2|{1, [[[-73.993217...|{1, [[[-73.9944, ...|{5, [[[-80.0, 35....|622236750717878271|622236723431276543|Manhattan| 114| 114|Greenwich Village...| false|622236750717878271|[01 03 00 00 00 0...|
 VTS|2009-01-03 01:55:00|2009-01-03 02:04:00| 5| 2.09| -73.993217| 40.72416| null| -73.9944| 40.74624| 7.7| 0.5| null| 0.0| 0.0| 8.2|{1, [[[-73.993217...|{1, [[[-73.9944, ...|{5, [[[-80.0, 35....|622236750717878271|622236723431276543|Manhattan| 144| 144| Little Italy/NoLiTa| false|622236750717878271|[01 03 00 00 00 0...|
 CMT|2009-12-07 10:36:06|2009-12-07 10:42:26| 1| 1.3| -73.99067| 40.73065| null| -73.975539| 40.741353| 6.1| 0.0| 0.5| 0.0| 0.0| 6.6|{1, [[[-73.99067,...|{1, [[[-73.975539...|{5, [[[-80.0, 35....|622236750715092991|622236723437502463|Manhattan| 113| 113|Greenwich Village...| false|622236750715092991|[01 03 00 00 00 0...|
 CMT|2009-12-07 10:36:06|2009-12-07 10:42:26| 1| 1.3| -73.99067| 40.73065| null| -73.975539| 40.741353| 6.1| 0.0| 0.5| 0.0| 0.0| 6.6|{1, [[[-73.99067,...|{1, [[[-73.975539...|{5, [[[-80.0, 35....|622236750715092991|622236723437

## MosaicFrame abstraction for simple indexing and joins

By wrapping our Spark DataFrames with `MosaicFrame`, we can simplify the join process. For example:

In [0]:
from mosaic import MosaicFrame

In [0]:
trips_mdf = MosaicFrame(trips, "pickup_geom")
neighbourhoods_mdf = MosaicFrame(geoJsonDF, "geometry")

In [0]:
(
  trips_mdf
  .set_index_resolution(10)
  .apply_index()
  .join(
    neighbourhoods_mdf
    .set_index_resolution(10)
    .apply_index()
  )
).show()

+---------+-------------------+-------------------+---------------+-------------+----------------+---------------+------------+-----------------+----------------+-----------+-----+-------+----------+------------+------------+--------------------+--------------------+--------------------+
vendor_id| pickup_datetime| dropoff_datetime|passenger_count|trip_distance|pickup_longitude|pickup_latitude|rate_code_id|dropoff_longitude|dropoff_latitude|fare_amount|extra|mta_tax|tip_amount|tolls_amount|total_amount| pickup_geom| dropoff_geom| bounds|
+---------+-------------------+-------------------+---------------+-------------+----------------+---------------+------------+-----------------+----------------+-----------+-----+-------+----------+------------+------------+--------------------+--------------------+--------------------+
 CMT|2009-10-12 19:35:21|2009-10-12 19:44:36| 1| 1.4| -74.000018| 40.720534| null| -73.995008| 40.734112| 6.5| 0.0| null| 0.0| 0.0| 6.5|{1, [[[-74.000018...|{1, [[[-73.995008...|{5, [[[-80.0, 35....|
 CMT|2010-09-02 07:37:40|2010-09-02 07:53:17| 1| 6.8| -74.016477| 40.70971| 1| -73.97509| 40.759392| 17.3| 0.0| 0.5| 1.0| 0.0| 18.8|{1, [[[-74.016477...|{1, [[[-73.97509,...|{5, [[[-80.0, 35....|
 CMT|2009-11-28 13:00:00|2009-11-28 13:01:57| 1| 0.5| -73.999666| 40.733384| null| -73.991751| 40.730273| 3.3| 0.0| 0.5| 0.0| 0.0| 3.8|{1, [[[-73.999666...|{1, [[[-73.991751...|{5, [[[-80.0, 35....|
 CMT|2009-11-30 07:49:13|2009-11-30 08:12:38| 1| 12.9| -74.007033| 40.734931| null| -73.87277| 40.774319| 29.7| 0.0| 0.5| 6.04| 0.0| 36.24|{1, [[[-74.007033...|{1, [[[-73.87277,...|{5, [[[-80.0, 35....|
 VTS|2009-01-03 01:55:00|2009-01-03 02:04:00| 5| 2.09| -73.993217| 40.72416| null| -73.9944| 40.74624| 7.7| 0.5| null| 0.0| 0.0| 8.2|{1, [[[-73.993217...|{1, [[[-73.9944, ...|{5, [[[-80.0, 35....|
 VTS|2009-11-09 17:21:00|2009-11-09 17:32:00| 1| 5.19| -74.010003| 40.707022| null| -73.963853| 40.75677| 13.3| 1.0| 0.5| 0.0| 0.0| 14.8|{1, [[[-74.010003...|{1, [[[-73.963853...|{5, [[[-80.0, 35....|
 VTS|2010-06-09 01:13:00|2010-06-09 01:27:00| 1| 8.09| -74.008697| 40.707335| 1| -73.949605| 40.78761| 19.7| 0.5| 0.5| 0.0| 0.0| 20.7|{1, [[[-74.008697...|{1, [[[-73.949605...|{5, [[[-80.0, 35....|
 VTS|2009-01-28 18:36:00|2009-01-28 18:50:00| 1| 2.92| -74.008823| 40.72089| null| -73.994055| 40.747588| 10.1| 1.0| null| 3.0| 0.0| 14.1|{1, [[[-74.008823...|{1, [[[-73.994055...|{5, [[[-80.0, 35....|
 CMT|2010-06-02 14:21:05|2010-06-02 14:28:43| 1| 5.0| -74.011653| 40.702604| 1| -73.971851| 40.749807| 12.5| 0.0| 0.5| 2.6| 0.0| 15.6|{1, [[[-74.011653...|{1, [[[-73.971851...|{5, [[[-80.0, 35....|
 VTS|2009-11-26 15:38:00|2009-11-26 15:47:00| 5| 2.36| -73.997208| 40.722108| null| -73.99647| 40.748185| 8.1| 0.0| 0.5| 2.0| 0.0| 10.6|{1, [[[-73.997208...|{1, [[[-73.99647,...|{5, [[[-80.0, 35....|
 CMT|2009-12-07 10:36:06|2009-12-07 10:42:26| 1| 1.3| -73.99067| 40.73065| null| -73.975539| 40.741353| 6.1| 0.0| 0.5| 0.0| 0.0| 6.6|{1, [[[-73.99067,...|{1, [[[-73.975539...|{5, [[[-80.0, 35....|
 CMT|2009-11-15 09:11:42|2009-11-15 09:26:34| 1| 3.7| -73.990065| 40.687341| null| -73.999566| 40.730219| 12.1| 0.0| 0.5| 0.0| 0.0| 12.6|{1, [[[-73.990065...|{1, [[[-73.999566...|{5, [[[-80.0, 35....|
 CMT|2010-06-27 17:27:25|2010-06-27 17:47:44| 2| 5.0| -74.008459| 40.712233| 1| -73.9516| 40.718877| 15.7| 0.0| 0.5| 3.0| 0.0| 19.2|{1, [[[-74.008459...|{1, [[[-73.9516, ...|{5, [[[-80.0, 35....|
 VTS|2009-01-30 21:05:00|2009-01-30 21:34:00| 1| 5.99| -73.994397| 40.694843| null| -73.921257| 40.685107| 18.9| 0.5| null| 0.6| 0.0| 20.0|{1, [[[-73.994397...|{1, [[[-73.921257...|{5, [[[-80.0, 35....|
 CMT|2009-01-14 07:43:53|2009-01-14 07:55:14| 2| 2.9| -74.007602| 40.725098| null| -73.977846| 40.751608| 9.3| 0.0| null| 0.0| 0.0| 9.3|{1, [[[-74.007602...|{1, [[[-73.977846...|{5, [[[-80.0, 35....|
 VTS|2010-05-11 16:33:00|2010-05-11 16:41:00| 1| 0.99| -74.011205| 40.715468| 1| -74.011072| 40.708882| 5.7| 1.0| 0.5| 0.0| 0.0| 7.2|{1, [[[-74.011205...|{1, [[[-74.011072...|{